#### Title: Multi-chamber real-time pose estimation  

##### Description:  
This notebook describes where in the DeepLabCut-Live! SDK and DeepLabCut-Live! GUI scripts to modify code to deliver random access targeting of multiple animals in individual chambers.  
The following 3 scripts are modified:  
(1) https://github.com/DeepLabCut/DeepLabCut-live-GUI/blob/master/dlclivegui/pose_process.py  
(2) https://github.com/DeepLabCut/DeepLabCut-live/blob/master/dlclive/utils.py  
(3) https://github.com/DeepLabCut/DeepLabCut-live/blob/master/dlclive/dlclive.py  

##### Attribution:  
This file is based on work originally found in the DeepLabCut Toolbox (deeplabcut.org, https://github.com/DeepLabCut/DeepLabCut-live, https://github.com/DeepLabCut/DeepLabCut-live-GUI) by A. & M. Mathis Labs. It was modified by Isobel Parkes (https://github.com/Isoparkes) and Liam E. Browne (https://github.com/lebrowne) for multi-animal assessment of nocifensive behaviours for Parkes et al 2024.  

##### Date:  
Created on: July 11, 2024  
Last updated: July 11, 2024  

##### Original License:  
This work includes modifications to code licensed under the GNU Lesser General Public License v3.0.  
For more information, see <https://www.gnu.org/licenses/lgpl-3.0.html>.  

##### Additional Contributions License:  
The modifications made by Isobel Parkes and Liam E. Browne are licensed under the MIT License. For more details, see <https://opensource.org/licenses/MIT>.  

#### (1) https://github.com/DeepLabCut/DeepLabCut-live-GUI/blob/master/dlclivegui/pose_process.py 

Insert below these two lines:  

```python
from dlclivegui import CameraProcess  
from dlclivegui.queue import ClearableQueue, ClearableMPQueue
```

In [ ]:
import cv2

x1 = 0 ### Modify according to ROI of all chambers
x2 = 0
y1 = 0
y2 = 0

all_rois = ((x1, y1), (x2, y2))
chamber0 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]] ### Add path to chamber ROI here and the below lines according to number of chambers (9 chambers used for Parkes et al)
chamber1 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber2 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber3 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber4 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber5 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber6 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber7 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber8 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]

chambers = [chamber0, chamber1, chamber2, chamber3, chamber4, chamber5, chamber6, chamber7, chamber8]
chambers_dict = {'0':[(), ()], ### Modify according to (x1,y2), (x2,y2) for each chamber ROI
'1':[(), ()],
'2':[(), ()],
'3':[(), ()],
'4':[(), ()],
'5':[(), ()],
'6':[(), ()],
'7':[(), ()],
'8':[(), ()]}

Insert within the *__CameraPoseProcess__* class below the following lines:  

```python
class CameraPoseProcess(CameraProcess):  
    """ Camera Process Manager class. Controls image capture, pose estimation and writing images to a video file in a background process.  
    
    Parameters  
    ----------  
    device : :class:`cameracontrol.Camera`  
        a camera object  
    ctx : :class:`multiprocess.Context`  
        multiprocessing context  
    """  
```

In [ ]:
running_stimulation = False ### Set up variables for psuedo-randomly selecting a chamber depending on the motion energy of an animal
stim = np.zeros(9)
idle_timestamps=np.empty(9); idle_timestamps.fill(1000000000000)
idle_threshold = 30000 ### User-defined (depending on the activity of the animals)
background_threshold = 10 ### User-defined (to remove background noise from motion energy calculation)
prev_frame = np.zeros((685,690),dtype=np.uint8) ### Modify according to dimensions of cropped chamber ROI
count = 0
t0 = 0
selected_chamber = None

Insert the custom methods within the *__CameraPoseProcess__* class after the constructor method:

```python
def __init__(self, device, ctx=mp.get_context("spawn")):
        """ Constructor method
        """

        super().__init__(device, ctx)
        self.display_pose = None
        self.display_pose_queue = ClearableMPQueue(2, ctx=self.ctx)
        self.pose_process = None
```

In [ ]:
def motion_in_roi(self,chambers, motion):

    chambers_motion = []
    for c in chambers:
        roi_motion = (c*motion).sum()
        chambers_motion.append(roi_motion)

    return chambers_motion

def idle_onset(self,chambers_motion, t, a):

    b = np.array((chambers_motion*1),dtype=np.int64)
    
    a[(a == 1000000000000) & (b == 1)] = t
    a[b == 0] = 1000000000000

    return a

def crop_to_selected_chamber(self, selected_chamber, chambers_dict, frame):

    crop_parameters = chambers_dict[selected_chamber]
    x1 = crop_parameters[0][0]
    x2 = crop_parameters[1][0]
    y1 = crop_parameters[0][1]
    y2 = crop_parameters[1][1]

    frame.base is frame
    full_view = frame.base

    cropped_chamber = full_view[y1:y2,x1:x2]
    
    return cropped_chamber.copy(), x1, y1

Insert in the method *___pose_loop__* within the *__CameraPoseProcess__* class:  

```python
def _pose_loop(self):
        """ Conduct pose estimation using deeplabcut-live in loop
        """
```

In [ ]:
run = True
write = False
frame_time = 0
pose_time = 0
end_time = time.time()
trial = False
trial_chamber = 0

Insert in the method *___pose_loop__* after the following lines:

```python
while run:

    ref_time = frame_time if self.opt_rate else end_time

    if self.frame_time[0] > ref_time:

        frame = self.frame
        frame_time = self.frame_time[0]
```

In [ ]:
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
motion = cv2.absdiff(self.prev_frame, frame)
motion = (motion>self.background_threshold)*motion
chambers_motion = self.motion_in_roi(chambers, motion)
chambers_motion = np.array(chambers_motion)<self.idle_threshold
t = time.time_ns() / 1000000000
idle_timestamps = self.idle_onset(chambers_motion, t, self.idle_timestamps)
ready_for_stim = (((t-idle_timestamps)>2) & ((t - self.stim)>10)) ### Animal has been stationary for 2 seconds (idle) and the chamber has not been selected in previous 10 seconds
if (ready_for_stim.any()): 
    if self.count == 0:
        chambers_ready = np.where(ready_for_stim)[0]
        self.selected_chamber = str(np.random.choice(chambers_ready,1)[0]) 
        self.t0 = t             
    if (t - self.t0) > 5:
        self.count = 0
        chambers_ready = np.where(ready_for_stim)[0]
        self.selected_chamber = str(np.random.choice(chambers_ready,1)[0])
        self.t0 = t
    self.prev_frame = frame
    if trial:
        self.selected_chamber = trial_chamber
    frame, x1, y1 = self.crop_to_selected_chamber(self.selected_chamber, chambers_dict, frame)  
    self.count += 1
else:
    self.count = 0
    self.prev_frame = frame
    frame = np.zeros((685,690),dtype=np.uint8) ### Modify according to dimensions of cropped chamber ROI
    x1 = 435 ### Modify according to dimensions of cropped chamber ROI
    y1 = 540 
    if not trial:
        self.selected_chamber = None

frame = frame.T
pose, trial = self.dlc.get_pose(frame, x1, y1, self.selected_chamber, frame_time=frame_time, record=write) ### This line replaces the original line: pose = self.dlc.get_pose(frame, frame_time=frame_time, record=write)
if trial:
    trial_chamber = self.selected_chamber


##### (2) https://github.com/DeepLabCut/DeepLabCut-live/blob/master/dlclive/utils.py 

Insert below the following lines:  

```python
import numpy as np
import warnings
from dlclive.exceptions import DLCLiveWarning
```

In [ ]:
x1 = 0 ### Modify according to ROI of all chambers
x2 = 0
y1 = 0
y2 = 0

all_rois = ((x1, y1), (x2, y2))
chamber0 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]] ### Add path to chamber ROI here and the below lines according to number of chambers (9 chambers used for Parkes et al)
chamber1 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber2 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber3 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber4 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber5 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber6 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber7 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]
chamber8 = np.load(r'path\ROI.npy')[all_rois[0][1]:all_rois[1][1],all_rois[0][0]:all_rois[1][0]]

chambers = [chamber0, chamber1, chamber2, chamber3, chamber4, chamber5, chamber6, chamber7, chamber8]
chambers_dict = {'0':[(), ()], ### Modify according to (x1,y2), (x2,y2) for each chamber ROI
'1':[(), ()],
'2':[(), ()],
'3':[(), ()],
'4':[(), ()],
'5':[(), ()],
'6':[(), ()],
'7':[(), ()],
'8':[(), ()]}

Insert the custom function below the function *__decode_fourcc__*

```python
def decode_fourcc(cc):
    """
    Convert float fourcc code from opencv to characters.
    If decode fails, returns empty string.
    https://stackoverflow.com/a/49138893
    Arguments:
        cc (float, int): fourcc code from opencv
    Returns:
         str: Character format of fourcc code

    Examples:
        >>> vid = cv2.VideoCapture('/some/video/path.avi')
        >>> decode_fourcc(vid.get(cv2.CAP_PROP_FOURCC))
        'DIVX'
    """
    try:
        decoded = "".join([chr((int(cc) >> 8 * i) & 0xFF) for i in range(4)])
    except:
        decoded = ""

    return decoded
```

In [ ]:
def original_coord_system(pose, x1, y1, selected_chamber):

    c1 = pose[:,0]+x1 
    c2 = pose[:,1]+y1
    c3 = pose[:,2]

    pose = np.vstack((c1, c2, c3)).T
    pose = np.insert(pose, 3, selected_chamber, axis=1)
    # print('chamber:', selected_chamber, 'x1:', x1, 'y1:', y1)

    return pose

#### (3) https://github.com/DeepLabCut/DeepLabCut-live/blob/master/dlclive/dlclive.py  

Insert as input parameters in the method *__get_pose__* within the *__DLCLive__* class

In [ ]:
x1=435, y1=540, selected_chamber=None ### Modify x1 and y1 according to cropped chamber ROI

### For example, def get_pose(self, frame=None, x1=435, y1=540, selected_chamber=None, **kwargs):

Insert below the following lines within the method *__get_pose__*–

```python
if self.dynamic_cropping is not None:
    self.pose[:, 0] += self.dynamic_cropping[0]
    self.pose[:, 1] += self.dynamic_cropping[2]
```

–replacing the original code lines:

```python
if self.processor:
    self.pose = self.processor.process(self.pose, **kwargs)

return self.pose
```

In [ ]:
if self.processor:
    self.pose = utils.original_coord_system(pose, x1, y1, selected_chamber)
    self.pose, self.trial = self.processor.process(self.pose, **kwargs)

return self.pose, self.trial